In [2]:
import numpy as np

# Load .npy file
file_path = "Paris_Instagram_TrafficMapSeries_20190316_20190531.npy"

with open('Paris_Instagram_TrafficMapSeries_20190316_20190531.npy', 'rb') as f:
        tensor_traffic_original = np.load(f)

cdims=[409,346]

# Group tiles to create 1km x 1km macro-tiles (each macro_tile contains 10 cells)
pad_x = 10 - cdims[0] % 10
pad_y = 10 - cdims[1] % 10

tensor_traffic_original = np.pad(tensor_traffic_original, ((0,0), (0,pad_x),(0,pad_y)), 'constant', constant_values = 0)
tensor_traffic = tensor_traffic_original.reshape(tensor_traffic_original.shape[0], int((cdims[0] + pad_x) / 10),
                                        10, int((cdims[1] + pad_y) / 10), 10).sum(axis=(2, 4))

Reshape the Data from (1848, 41, 35) to (1848, 2, 41, 35)

In [12]:
display(tensor_traffic.shape)

tensor_traffic_reshape = np.reshape(tensor_traffic, (1848, 1, 41, 35))

tensor_traffic_reshape_2 = np.concatenate((tensor_traffic_reshape, tensor_traffic_reshape), axis=1)
#tensor_traffic_reshape_2=tensor_traffic_reshape
#tensor_traffic_reshape_2 = tensor_traffic

display(tensor_traffic_reshape_2.shape)



(1848, 41, 35)

(1848, 2, 41, 35)

import a dataset to extract the time date_arr and take the first 1848 elements same dimensionality of the network dataset

In [13]:

import h5py
f = h5py.File('../../BikeNYC/NYC14_M16x8_T60_NewEnd.h5')
for ke in f.keys():
    print(ke, f[ke].shape)

date_arr = np.array(f['date'][:1848])

data_arr_b = np.array(f['data'])
date_arr_b = np.array(f['date'])

data (4392, 2, 16, 8)
date (4392,)


merge the 2 dataset and save it in h5 format

In [14]:
import h5py
# Create an h5py file
with h5py.File('Paris_Instagram_TrafficMapSeries_20190316_20190531_REDUCED.h5', 'w') as hf:
    # Create a dataset inside the file
    hf.create_dataset('data', data=tensor_traffic_reshape_2)
    hf.create_dataset('date', data=date_arr)
    
    
f = h5py.File('Paris_Instagram_TrafficMapSeries_20190316_20190531_REDUCED.h5')
for ke in f.keys():
    print(ke, f[ke].shape)
    
data_arr = np.array(f['data'])
date_arr = np.array(f['date'])

data (1848, 2, 41, 35)
date (1848,)
